<a href="https://colab.research.google.com/github/inspire007/KaggleCompetitions/blob/main/Natural_Language_Processing_with_Disaster_Tweets/Natural_Language_Processing_with_Disaster_Tweets_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-h

In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
from gensim.models import KeyedVectors

word2vecModel = KeyedVectors.load('/content/drive/MyDrive/word2vec-google-news.model')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import re

In [3]:
dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')

In [17]:
def text_preprocessing(sentence):
  sentence = re.sub('[^a-zA-Z]', ' ', sentence)
  words = [w for w in sentence.lower().split() if w in word2vecModel]
  if not words:
    return np.zeros(word2vecModel.vector_size)
  return np.mean([word2vecModel[w] for w in words], axis=0)

In [18]:
X = np.array([text_preprocessing(s) for s in dataset['text']])
y = dataset['target'].values

X_test = np.array([text_preprocessing(s) for s in test_dataset['text']])

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [20]:
classifier = SVC(kernel='rbf')
cvs = cross_val_score(estimator=classifier, X=X, y=y, cv=5)

In [21]:
print('Accuracy score: ', cvs.mean(), '\n')

Accuracy score:  0.7964032879983918 



In [22]:
classifier.fit(X,y)
y_predict = classifier.predict(X_test)

In [23]:
ids = test_dataset['id'].values

np.savetxt('output.csv', np.column_stack((ids,y_predict)), delimiter=',', comments='', header='id,target', fmt=['%d', '%d'])